# Assignment 3 - Transliteration

### Setup [do not change]

In [1]:
!git clone https://github.com/vicky-xef/eth-nlp-f24-hw3.git

import os
os.chdir('eth-nlp-f24-hw3')

!pip install -e .

fatal: destination path 'eth-nlp-f24-hw3' already exists and is not an empty directory.


Obtaining file:///C:/UZH/NLP_ETH/Assignment3_WFSTs/eth-nlp-f24-hw3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: rayuela
    Found existing installation: rayuela 0.1
    Uninstalling rayuela-0.1:
      Successfully uninstalled rayuela-0.1
  Running setup.py develop for rayuela


In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC' -O 'test_cases.pkl'

'wget' is not recognized as an internal or external command,
operable program or batch file.
'id' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from rayuela.base.semiring import Semiring, Real, Tropical
from rayuela.base.symbol import Sym, ε, ε_1, ε_2
from rayuela.fsa.fsa_classes import get_epsilon_filter

from rayuela.fsa.fst import FST
from rayuela.fsa.state import State, PairState

import dill
import numpy as np
from math import exp, log

from itertools import product

from frozendict import frozendict

### Subquestion a) - Log Semiring

In [4]:
class Log(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        # TODO: implement Kleene star
        if self.score < 0:
            return Log(log(1/(1-exp(self.score))))
        else:
            Log(np.inf)

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        # TODO: implement log addition
        # For numerical stability you should use the using the log-sum-exp trick
        # https://en.wikipedia.org/wiki/LogSumExp
        return Log(np.logaddexp(self.score, other.score))

    def __mul__(self, other):
        # TODO: implement log multiplication
        return Log(self.score + other.score)

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

# TODO: implement the log semiring one and zero
Log.zero = Log(-np.inf)
Log.one = Log(0)


Log.idempotent = False
Log.cancellative = True

### Before moving to the next subquestion, we give some examples of WFSTs to illustrate how to use the library `rayuela`.

In [5]:
# A simple language model encoded by a WFST

sem = Real # the Real semiring
fst = FST(sem) # a WFST in the Real semiring

# add transitions
fst.add_arc(State(0), Sym('formal'), Sym('formal'), State(1), Real(0.2))
fst.add_arc(State(0), Sym('natural'), Sym('natural'), State(1), Real(0.3))
fst.add_arc(State(0), Sym('learning'), Sym('learning'), State(2), Real(0.2))
fst.add_arc(State(0), Sym('data'), Sym('data'), State(3), Real(0.3))

fst.add_arc(State(1), Sym('language'), Sym('language'), State(2), Real(0.6))
fst.add_arc(State(1), Sym('languages'), Sym('languages'), State(4), Real(0.4))

fst.add_arc(State(2), Sym('is'), Sym('is'), State(5), Real(1.0))

fst.add_arc(State(5), Sym('fun'), Sym('fun'), State(6), Real(1.0))

fst.add_arc(State(3), Sym('is'), Sym('is'), State(5), Real(0.5))
fst.add_arc(State(3), Sym('are'), Sym('are'), State(5), Real(0.5))

fst.add_arc(State(4), Sym('are'), Sym('are'), State(5), Real(1.0))

# set initial weights
# if not set, it defaults to semiring zero
fst.set_I(State(0), Real(1.0))
# set final weights
fst.set_F(State(6), Real(1.0))

# visualize
fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

In [6]:
# A simple WFSA for transliteration

sem = Real # semiring
fst = FST(sem) # initialize WFST in the chosen semiring
one, zero = sem.one, sem.zero # semiring one and zero

# add transitions
fst.add_arc(State(0), Sym('d'), Sym('data'), State(1), Real(0.5))
fst.add_arc(State(0), Sym('d'), Sym('dew'), State(5), Real(0.5))

fst.add_arc(State(1), Sym('ey'), Sym('ÃŽÂµ'), State(2), Real(0.5))
fst.add_arc(State(1), Sym('ae'), Sym('ÃŽÂµ'), State(2), Real(0.5))

fst.add_arc(State(2), Sym('t'), Sym('ÃŽÂµ'), State(3), Real(0.7))
fst.add_arc(State(2), Sym('dx'), Sym('ÃŽÂµ'), State(3), Real(0.3))

fst.add_arc(State(3), Sym('ax'), Sym('ÃŽÂµ'), State(4), one)

fst.add_arc(State(5), Sym('uw'), Sym('ÃŽÂµ'), State(6), one)

# initial and final weights
fst.set_I(State(0), one)
fst.set_F(State(4), one)
fst.set_F(State(6), one)

fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion b) - String Transducer

In [7]:
def string_fst(s, semiring=Log):
    """ Takes a string as input and a semiring
    and returns a transducer encoding of the
    string
    """
    fst = FST(semiring)

    # TODO: add arcs, initial and final weights
    for i, char in enumerate(s):
        fst.add_arc(State(i), Sym(char), Sym(char), State(i+1), semiring(0.5))
        
    # Set initial and final states
    fst.set_I(State(0), semiring.one)
    fst.set_F(State(len(s)), semiring.one)
    return fst

In [8]:
# you can visualise the transducer
fst = string_fst("abc", semiring=Real)
fst

WFSA(4 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion c) - Edit Distance Transducer

As we will not learn the weights on the arcs, you can set them to some pre-defined constants. You should choose the weights such that the pathsum that needs to be computed in e) does not diverge (this is caused by the weights of the self-loops in the transducer).

In [9]:
def edit_distance_fst(source_alphabet, target_alphabet, semiring=Log):
    """ Takes as input two sets of characters (source and target
    alphabets) and a semiring and returns an edit-distance transducer
    """

    fst = FST(semiring)

    # Deletion initial case -> No matter source char we can output epsilon
    for s in source_alphabet:
        fst.set_arc(State(0), Sym(s), ε, State(0), semiring.one)
        
    # Subtitution -> For each char in source we output a different char in target
    for j, t in enumerate(target_alphabet, start=1):
        for s in source_alphabet:
            fst.set_arc(State(0), Sym(s), Sym(t), State(j), semiring.one)
        # Insertion -> We get epsilon and return t
        fst.set_arc(State(0), ε, Sym(t), State(j), semiring.one)
        
    # Deletion for each state j
    for j, t in enumerate(target_alphabet, start=1):
        for s in source_alphabet:
            fst.set_arc(State(j), Sym(s), ε, State(j), semiring.one)
            
    # Subtitution among states j
    for j1, t1 in enumerate(target_alphabet, start=1):
        for j2, t2 in enumerate(target_alphabet, start=1):
            if j1!=j2:
                for s in source_alphabet:
                    fst.set_arc(State(j1), Sym(s), Sym(t2), State(j2), semiring.one)
                # Insertion case between target states
                fst.set_arc(State(j1), ε, Sym(t2), State(j2), semiring.one)
        

    # Initial and final states:
    fst.set_I(State(0), semiring.one)
    # At the same time, the initial state can be an end state
    fst.set_F(State(0), semiring.one)
    
    # All states j can be a final state
    for j, t in enumerate(target_alphabet, start=1):
        fst.set_F(State(j), semiring.one)
                             
    return fst

In [10]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Real)
fst

WFSA(3 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion d) - Composition

We provide some composition test cases for you to inspect.

In [14]:
# Important, since the cwd changed the directory to the cloned folder, I added my pkl file in the cloned folder

with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)

In [15]:
# FST 1
examples[0]['fst1']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [16]:
# FST 2
examples[0]['fst2']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [17]:
# composition
examples[0]['composition']['fst']

WFSA(3 states, <class 'rayuela.base.semiring.Real'>)

In [18]:
# semiring
examples[0]['semiring']

rayuela.base.semiring.Real

In [163]:
vis = None

def compose(fst1, fst2):
    """ Takes as input two transducers and returns
    their composition
    """
    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)

    # TODO: implement composition
    # Implementation of the accessible composition provided in the lecture
    # Use copies to avoid modifying the original Transducers
    fst1_copy = fst1.augment_epsilon_transitions(idx=1)
    fst2_copy = fst2.augment_epsilon_transitions(idx=2)
    
    epsilon_filter = get_epsilon_filter(fst1.R, fst1.Sigma.union(fst2.Sigma))
    
    stack = []
    visited = set()
    
    # Add initial states to stack
    for s1, w1 in fst1_copy.I:
        for s2, w2 in fst2_copy.I:
            for es, ew in epsilon_filter.I:
                comp_state = PairState(s1,s2)
                comp_state = PairState(comp_state, es)
                stack.append((s1, s2, es))
                visited.add((s1, s2, es))
            
                # \lambda_T <- \lambda_1(q1) x \lambda_2(q2)
                comp_fst.set_I(comp_state, w1*w2*ew)
            
    while len(stack) > 0:
        q1, q2, ef_state = stack.pop()
        comp_state = PairState(q1,q2)
        comp_state = PairState(comp_state, ef_state)
        # valid_tranisitions = []
        for transition in fst1_copy.arcs(q1):
            a, b = transition[0]
            q1_prima = transition[1]
            w1 = transition[2]
            
            for transition2 in fst2_copy.arcs(q2):
                c, d = transition2[0]
                q2_prima = transition[1]
                w2 = transition[2]
                
                for transition_e in epsilon_filter.arcs(ef_state):
                    ef_in, ef_out = transition_e[0]
                    ef_prima = transition_e[1]
                    ef_w = transition_e[2]
                
                    #if b == c or (b in {ε, ε_1} and c in {ε, ε_2}):
                    if b==c or (b==ef_in and c==ef_out):
                        next_comp_state = PairState(q1_prima, q2_prima)
                        next_comp_state = PairState(next_comp_state, ef_prima)
                        comp_fst.set_arc(comp_state, a, d, next_comp_state, w1*w2*ef_w)
                    
                    if (q1_prima, q2_prima, ef_prima) not in visited:
                        stack.append((q1_prima, q2_prima, ef_prima))
                        visited.add((q1_prima, q2_prima, ef_prima))
                        
# if epsilon_filter(b,c,comp_state) != State("⊥"):
#                     valid_transitions.append(((q1_prima, (a,b), w1), (q2_prima, (c,d), w2)))
                    
#         for (start, (a,b), w1), (end, (c,d), w2) in valid_transitions:
#             comp_fst.set_arc(PairState(q1,q2), a, d, PairState(start,end), w1*w2)
            
#             qfp = epsilon_filter(b, c, qf)

#             if (start,end,qfp) not in visited:
#                 stack.append((start,end,qfp))
#                 visited.add((start,end))
                        
    
#     for pair in comp_fst.Q:
#         wf = fst1_copy.ρ[pair.state1] * fst2_copy.ρ[pair.state2]
#         if wf != fst1_copy.R.zero:
#             comp_fst.add_F(pair, wf)

    for (q1, w1) in fst1_copy.F:
        for (q2, w2) in fst2_copy.F:
            for ef_state, ef_w in epsilon_filter.F:
                comp_state = PairState(q1,q2)
                comp_state = PairState(comp_state, ef_state)
                comp_weight = w1*w2*ef_w
                if comp_weight != comp_fst.R.zero:
                    comp_fst.set_F(comp_state, comp_weight)
            
    
    
    
    return comp_fst

In [164]:
vis

In [165]:
compose(examples[0]['fst1']['fst'],examples[0]['fst2']['fst'])

WFSA(6 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion e) - Lehmann's Algorithm

In [166]:
class Pathsum:
    def __init__(self, fsa):

        # basic FSA stuff
        self.fsa = fsa
        self.R = fsa.R
        self.N = self.fsa.num_states

        # state dictionary
        self.I = {}
        for n, q in enumerate(self.fsa.Q):
            self.I[q] = n

        # lift into the semiring
        self.W, self.W_dict, self.alpha, self.beta = self.lift()


    def lift(self):
        """ creates the weight matrix, initial weight vector,
        final weight vector from the automaton """
        W = self.R.zeros((self.N, self.N))
        W_dict = {a: {} for a in self.fsa.Sigma}
        alpha = self.R.zeros((self.N,))
        beta = self.R.zeros((self.N,))
        for a in self.fsa.Sigma:
            for b in self.fsa.Delta:
                W_dict[a][b] = self.R.zeros((self.N, self.N))
        for p in self.fsa.Q:
            alpha[self.I[p]] += self.fsa.λ[p]
            beta[self.I[p]] += self.fsa.ρ[p]
            for (a, b), q, w in self.fsa.arcs(p):
                W[self.I[p], self.I[q]] += w
                W_dict[a][b][self.I[p], self.I[q]] = w
        return W, W_dict, alpha, beta

    def lehmann(self):
        """
        Lehmann's (1977) algorithm.
        """
        # Implementation following the algorithm given in lecture
        R = self.W
        for j in range(self.N):
            for i in range(self.N):
                for k in range(self.N):
                    R[i, k] = R[i,k] + R[i,j]*R[j,j].star()*R[j,k]
                    
        for i in range(self.N):
            R[i,i] = R[i,i] + self.R.one
        
        return R

    def _iterate(self, K):
        P = self.R.diag(self.N)
        for n in range(K):
            P += self.W @ P
        return P

    def _fixpoint(self, K=200):
        if self.fsa.R.idempotent:
            return self._iterate(self.fsa.num_states)

        diag = self.R.zeros((self.fsa.num_states, self.fsa.num_states))
        for n in range(self.fsa.num_states):
            diag[n, n] = self.fsa.R.one
        P_old = diag

        for _ in range(K):
            P_new = diag + self.W @ P_old
            P_old = P_new

        return P_old

    def fixpoint(self):

        P = self._fixpoint()
        W = {}

        for p in self.fsa.Q:
            for q in self.fsa.Q:
                if p in self.I and q in self.I:
                    W[p, q] = P[self.I[p], self.I[q]]
                elif p == q:
                    W[p, q] = self.R.one
                else:
                    W[p, q] = self.R.zero

        return frozendict(W)

In [167]:
pathsum = Pathsum(examples[0]['fst1']['fst'])
pathsum.fixpoint()

frozendict.frozendict({(0, 0): 1.0, (0, 1): 0.25, (1, 0): 0.0, (1, 1): 1.146788990825688})

In [168]:
pathsum.lehmann()

array([[1.0, 0.25],
       [0.0, 1.146788990825688]], dtype=object)

### Subquestion f) - Normalizer

We provide some test cases for you to inspect.

In [ ]:
examples[0]['fst1']['Z'], examples[0]['fst2']['Z'], examples[0]['composition']['Z']

In [171]:
def normalizer(fst):
    """ Takes as input a finite-state transducer and
    returns its normalizer
    """

    pathsum = Pathsum(fst)
    # According to the lecture Z(x) = sum_{qn,qm \in Q} \lambda(qn)(W*)\rho(qm)
    Z = fst.R.zero
    R = pathsum.lehmann()
    
    for n, qn in enumerate(fst.Q):
        for m, qm in enumerate(fst.Q):
            Z = Z + fst.λ[qn]*R[n,m]*fst.ρ[qm]

    return Z

In [172]:
normalizer(examples[0]['fst1']['fst']), normalizer(examples[0]['fst2']['fst']), normalizer(examples[0]['composition']['fst'])

(0.00138425, 0.000997195294118, 7.08067346e-07)

### Subquestion g) - Log-likelihood

In [173]:
def log_likelihood(source_alphabet, target_alphabet, source_str, target_str):
    """ Takes as input the source and target alphabets (sets of characters)
    and a pair of source and target strings and returns the log-likelihood
    """

    # TODO: implement log-likelihood
    # \sum_{i=1}^I score(yi,xi)-logZ(xi)
    
    # score(yi, xi) is given by the composed transducer
    
    # Build transducers
    src_fst = string_fst(source_str)
    target_fst = string_fst(target_str)
    edit_dist = edit_distance_fst(source_alphabet, target_alphabet)
    
    # Composition...
    comp1 = compose(src_fst, edit_dist)
    comp2 = compose(comp1, target_fst)
    """
    Recall that p (y | x) sums over all possible alignments of x to y. This probability
    can be computed as the pathsum of the composition T x ◦ T ◦ T y, divided by the
    pathsum of T x ◦ T (the normalizing constant).
    """
    score_theta = np.log(normalizer(comp2).score)
    log_z = np.log(normalizer(comp1).score)
    
    return score_theta-log_Z
    

### Subquestion h) - Decoding

In [ ]:
def to_semiring(fst, semiring):
    """ Lifts an WFST into a different semiring
    """
    oone, one = fst.R.one, semiring.one
    nfst = FST(semiring)

    # initial weights
    for q, w in fst.I:
        if w == oone:
            nfst.set_I(q, one)
        else:
            nfst.set_I(q, semiring(w.score))

    # final weights
    for q, w in fst.F:
        if w == oone:
            nfst.set_F(q, one)
        else:
            nfst.set_F(q, semiring(w.score))

    # arcs
    for p in fst.Q:
        for (a, b), q, w in fst.arcs(p):
            if w == oone:
                nfst.add_arc(p, a, b, q, one)
            else:
                nfst.add_arc(p, a, b, q, semiring(w.score))

    return nfst

In [174]:
"""
We need to implement a different semiring
If we replace the probability or log-semiring with the arctic semiring
〈R≤0, max, +, −∞, 0〉 on a trained instance on the WFST, we can find the
(probability of) the most probable transliteration y for a given input string x!
"""

class Arctic(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        # TODO: implement Kleene star
        return self.one

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        return Arctic(max(self.score, other.score))

    def __mul__(self, other):
        # Remember idempotent property a+a = a and 0-closed 1+a=1...
        if other is self.one: # a+0 = a
            return self
        if self is self.one:
            return other
        if self is self.zero or other is self.zero:
            return self.zero # somthing not infty + -infty -> -infty
        return Arctic(self.score+other.score)

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

# TODO: implement the log semiring one and zero
Arctic.zero = Arctic(-np.inf)
Arctic.one = Arctic(0.0)


Arctic.idempotent = True
Arctic.cancellative = True
    

In [175]:
def decode(in_str, edit_distance_fst):
    """ Gets as input a source string and an
    edit-distance transducer and returns the weight of the most
    probable target string
    """
    # TODO: implement
    in_fst = string_fst(in_str)
    comp = compose(in_fst, edit_distance_fst)
    
    # Raise to different semiring
    comp = to_semiring(comp, Arctic)
    
    return normalizer(comp)